## MYSQL 관련 모듈 설치
 
- conda install MySQL-python  or
- pip install MySQL-python


- wget http://dev.mysql.com/get/Downloads/Connector-Python/mysql-connector-python-cext_2.1.3-1ubuntu14.04_amd64.deb
- sudo dpkg -i  mysql-connector-python-*.deb
- or
- conda install mysql-connector-python


In [5]:
# MySQL-python 을 이용할때

import MySQLdb as mdb
import sys

try:
    con = mdb.connect('node02', 'deepbio', 'deepbio1234', 'deepbioDB');
     
    cur = con.cursor()
    cur.execute("SELECT VERSION()")

    ver = cur.fetchone()
    
    print "Database version : %s " % ver
    
except mdb.Error, e:
  
    print "Error %d: %s" % (e.args[0],e.args[1])
    sys.exit(1)
    
finally:    
        
    if con:    
        con.close()

Database version : 5.5.47-MariaDB-1~trusty-log 


In [6]:
# mysql-connector-python을 이용할때

import mysql.connector as mariadb

try:
    mariadb_connection = mariadb.connect(host='node02', user='deepbio', password='deepbio1234', database='deepbioDB')
    cursor = mariadb_connection.cursor()
    cursor.execute("SELECT VERSION()")
    ver = cursor.fetchone()
    print "Database version : %s " % ver
    
except mariadb.Error as error:
    print("Error: {}".format(error))
    
mariadb_connection.commit()

mariadb_connection.close()    

Database version : 5.5.47-MariaDB-1~trusty-log 


### DB 와 HBase 스키마 정의
```
-- ##############################################################
-- DB 스키마 정의
-- ##############################################################

CREATE SCHEMA IF NOT EXISTS `deepbioDB` DEFAULT CHARACTER SET utf8 COLLATE utf8_general_ci ;
USE `deepbioDB` ;

DROP TABLE IF EXISTS file_manifest ;

CREATE TABLE IF NOT EXISTS file_manifest (
  `id`       MEDIUMINT NOT NULL AUTO_INCREMENT,
  `disease` VARCHAR(10) NOT NULL,
  `platform_type` VARCHAR(50) NOT NULL,
  `center` VARCHAR(10) NOT NULL,
  `platform` VARCHAR(50) NOT NULL,
  `level` VARCHAR(10) NOT NULL,
  `barcode` VARCHAR(50) NOT NULL,
  `file_name` VARCHAR(512) NOT NULL,
  `reg_dt` TIMESTAMP DEFAULT CURRENT_TIMESTAMP  ,
  PRIMARY KEY (`id`),
  index file_manifefile_annotationsst_barcode_idx(barcode) 
)
ENGINE = InnoDB;


select count(*) from file_manifest;
select * from file_manifest limit 10;
select count(*) from file_annotations;

DROP TABLE IF EXISTS file_annotations ;

CREATE TABLE IF NOT EXISTS file_annotations (
  `id`       MEDIUMINT NOT NULL ,
  `disease` VARCHAR(10) NOT NULL,
  `item_type` VARCHAR(50) NOT NULL,
  `item_barcode` VARCHAR(50) NOT NULL,
  `item_uuid` VARCHAR(50) NOT NULL,
  `classification` VARCHAR(10) NOT NULL,
  `category` VARCHAR(50) NOT NULL,
  `annotator` VARCHAR(50) NOT NULL,
  `date_created` VARCHAR(20) NOT NULL,
  `annotation` VARCHAR(4096) NOT NULL,
  `reg_dt` TIMESTAMP DEFAULT CURRENT_TIMESTAMP  ,
  PRIMARY KEY (`id`),
  index file_annotations_barcode_idx(item_barcode) ,
  index file_annotations_uuid_idx(item_uuid) ,
  index file_annotations_date_created_idx(date_created) 
)
ENGINE = InnoDB;


-- ##############################################################
-- HBASE 스키마 정의
-- ##############################################################

disable 'mRNA'
drop 'mRNA'
create 'mRNA', {NAME => 'manifest' },  {NAME=>'data', COMPRESSION=>'SNAPPY'}


```

## CSV와 Tab을 구분된 파일 읽어서 DB에 넣는 모듈 

- 다른 장비에 있는 파일을 가지고 오기.

```
scp  spark@node03:/home/spark/tcga/01/file_annotations.txt  /home/deepbio/note/work/tcga/node03_01_file_annotations.txt
scp  spark@node03:/home/spark/tcga/02/file_annotations.txt  /home/deepbio/note/work/tcga/node03_02_file_annotations.txt
scp  spark@node03:/home/spark/tcga/03/file_annotations.txt  /home/deepbio/note/work/tcga/node03_03_file_annotations.txt
```

```
scp  spark@node03:/home/spark/tcga/01/file_manifest.txt /home/deepbio/note/work/tcga/node03_01_file_manifest.txt
scp  spark@node03:/home/spark/tcga/02/file_manifest.txt /home/deepbio/note/work/tcga/node03_02_file_manifest.txt
scp  spark@node03:/home/spark/tcga/03/file_manifest.txt /home/deepbio/note/work/tcga/node03_03_file_manifest.txt
```

In [5]:
! ls -al /home/deepbio/note/work/tcga/

total 6044
drwxrwxr-x 3 deepbio deepbio    4096 Dec 24 14:43 .
drwxr-xr-x 7 deepbio deepbio    4096 Dec 24 11:01 ..
drwxrwxr-x 7 deepbio deepbio    4096 Dec 24 14:12 01
-rw-r--r-- 1 deepbio deepbio   41208 Dec 24 14:43 node03_01_file_annotations.txt
-rw-r--r-- 1 deepbio deepbio 1812426 Dec 24 14:24 node03_01_file_manifest.txt
-rw-r--r-- 1 deepbio deepbio   23257 Dec 24 14:43 node03_02_file_annotations.txt
-rw-r--r-- 1 deepbio deepbio 2006214 Dec 24 14:40 node03_02_file_manifest.txt
-rw-r--r-- 1 deepbio deepbio  120974 Dec 24 14:43 node03_03_file_annotations.txt
-rw-r--r-- 1 deepbio deepbio 2158818 Dec 24 14:41 node03_03_file_manifest.txt


## mRNA 데이터파일을 HBASE에 넣는 모듈 

In [59]:
##
## file_manifest.txt 파일과 file_annotations.txt 파일을 DB에 upload하는 모듈
## upload_db.py

DB_HOST = 'node02'
DB_USER = 'deepbio'
DB_PW = 'deepbio1234'
DB_NAME = 'deepbioDB'
BULK_INSERT_SIZE = 500

SEPARATOR = '\t'
INT_COLUMNS = ['id']


# 해더를 파싱해서 DB columns 리스트로 변경함.
# 모두 소문자 처리, 공백은 언더바로 변경함.
def divHeader( line,  sep = '\t') :
    if None == line : return None

    cols = line.split( sep )
    for idx in range(len(cols)):
        cols[idx] = cols[idx].lower().replace(' ', '_')
    return cols

#   INSERT 문을 만들어주는 함수
#  예) INSERT INTO tablename ( col1, col2, col3, col4 )  values ( 'val1', 'val2', 'val3' )
def makeInsertSQL( tablename, columns, rows ) :    
    if  len( columns ) >  len( rows )   :  return None
    
    from cStringIO import StringIO
    insertSQL = StringIO()
    insertSQL.write('INSERT INTO ')
    insertSQL.write( tablename )
    insertSQL.write(' ( ')
    for idx in range(len(columns)) : 
        insertSQL.write( columns[idx] )
        if( len(columns)-1 > idx ) : insertSQL.write( ', ' )
    insertSQL.write(' ) ')

    insertSQL.write(' VALUES ( ')
    for idx in range(len(columns)) : 
        if columns[idx] not in  INT_COLUMNS : insertSQL.write("'")    
        insertSQL.write( rows[idx] )
        if columns[idx] not in  INT_COLUMNS : insertSQL.write("'")
        if( len(columns)-1 > idx ) : insertSQL.write( ', ' )    
    insertSQL.write(' ) ')    
    
    return insertSQL.getvalue()    

# 
# 데이터 파일을 읽어서 DB에 저장하는 함수.
#
def fileReadAndInsert( dbconnection, cursor , fileName, tablename ) : 
    headers = []
    rows = []
    isHeader = True
    sqlCount = 0
    commitCount = 0    

    fhand = open(  fileName  )
    for line in fhand:
        if True == isHeader : 
            headers = divHeader( line, SEPARATOR )
            isHeader = False
            continue

        rows = line.strip().split( SEPARATOR )
        
        #print 'headers', len( headers )
        #print ', rows', len( rows )        
        #break
        #if  len( headers ) >  len( rows )   : 
        #    print line
        #    continue
        
        sql = makeInsertSQL(tablename, headers, rows )
        try:
            if  None !=  cursor and  None != sql  :  cursor.execute( sql )
        except mariadb.Error as error:
            print("Error: {}".format(error))

        sqlCount += 1
        if sqlCount >= BULK_INSERT_SIZE : 
            if  None !=  dbconnection : dbconnection.commit()
            commitCount += sqlCount
            print "commit count=", commitCount
            sqlCount = 0

    if  None !=  dbconnection : dbconnection.commit()            
    commitCount += sqlCount
    print "commit count=", commitCount    
    return commitCount


import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor = mariadb_connection.cursor()
cursor.execute('SET autocommit = 0')

#mariadb_connection = None
#cursor = None

#TABLE_NAME = 'file_manifest'
#DATA_FILES = [ '/home/deepbio/note/work/tcga/node03_01_file_manifest.txt'
#             ,'/home/deepbio/note/work/tcga/node03_02_file_manifest.txt'
#             ,'/home/deepbio/note/work/tcga/node03_03_file_manifest.txt' ]

#TABLE_NAME = 'file_annotations'
#DATA_FILES = [ '/home/deepbio/note/work/tcga/node03_01_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node03_02_file_annotations.txt'
#             ,'/home/deepbio/note/work/tcga/node03_03_file_annotations.txt' ]


TABLE_NAME = 'file_manifest'
DATA_FILES = [ '/home/deepbio/note/work/tcga/01/file_manifest.txt' ]

totalCommit = 0
for datafile in  DATA_FILES : 
    totalCommit += fileReadAndInsert( mariadb_connection, cursor , datafile, TABLE_NAME )

if  None !=  mariadb_connection : mariadb_connection.commit()
if  None !=  mariadb_connection : mariadb_connection.close()

print "Total INSERT=", totalCommit

commit count= 500
commit count= 1000
commit count= 1500
commit count= 2000
commit count= 2500
commit count= 3000
commit count= 3500
commit count= 4000
commit count= 4500
commit count= 5000
commit count= 5500
commit count= 6000
commit count= 6274
Total INSERT= 6274


- 설치 방법
```
(envname) $ pip install happybase
```

- hbase는 thrift server가 구동이 되어야함.

```
bin/hbase-daemon.sh start thrift  
```


In [18]:
import happybase
connection = happybase.Connection('node03')
table = connection.table('mRNA')

In [ ]:
#
# DB에 등록된 정보를 이용해서 물리적인 파일을 HBASE에 올리는 모듈 
# upload_hbase.py

DB_HOST = 'node02'
DB_USER = 'deepbio'
DB_PW = 'deepbio1234'
DB_NAME = 'deepbioDB'
QUERY_FETCH_SIZE = 20
BASE_PATH = '/home/deepbio/note/work/tcga/01/'

## Disease Platform Type   Center  Platform        Level   Barcode File Name
## COAD    RNASeqV2        UNC     IlluminaGA_RNASeqV2     Level_3 TCGA-AA-3814-01A-01R-0905-07    unc.edu.51364582-19a3-4d49-8a15-2f7f42860e39.1589076.rsem.isoforms.results
## => /COAD/RNASeqV2/UNC__IlluminaHiSeq_RNASeqV2/Level_3
def processDBData( cursor, pool ) :
    
    fullPath = ''
    for ID ,disease, platform_type, center, platform, level, barcode, file_name  in cursor:
        fullPath =  "{}/{}/{}__{}/{}/{}".format(disease, platform_type, center, platform, level, file_name) 
        contents = readFile( BASE_PATH + fullPath ) 
        if None == contents : continue 
        uploadFileToHBase( pool, disease, platform_type, center, platform, level, barcode, file_name, contents )


##  파일의 전체 내용을 String으로 읽어들임.
def readFile( fname ) :
    if True != os.path.exists(fname) : 
        print 'read not ' + fname
        return None

    content = None
    with open(fname, 'r') as content_file:
        content = content_file.read()

    return content    
    
## HBASE로 데이터를 업로드함.
def uploadFileToHBase( pool, disease, platform_type, center, platform, level, barcode, file_name,  contents ) :
    with pool.connection() as connection:
        table = connection.table('mRNA')
        print 'put rowkey=' + barcode
        table.put(barcode, {'manifest:disease': disease,
                       'manifest:platform_type': platform_type,
                       'manifest:center': center,
                       'manifest:platform': platform,
                       'manifest:level': level,
                       'manifest:barcode': barcode,
                       'manifest:file_name': file_name,
                       'data:contents': contents })
        
        
import happybase
pool = happybase.ConnectionPool(size=3, host='node03')    
    
import mysql.connector as mariadb
mariadb_connection = mariadb.connect(host=DB_HOST, user=DB_USER, password=DB_PW, database=DB_NAME)
cursor = mariadb_connection.cursor()


cursor.execute("SELECT count(*) as totalcount FROM file_manifest ")
totalCount = cursor.fetchone() 
#print "totalCount=", totalCount

offset = 0;
pageCount = totalCount[0] / QUERY_FETCH_SIZE 
if totalCount[0] % QUERY_FETCH_SIZE  > 0 : 
    pageCount += 1

# debug용
#pageCount = 1
    
for idx in range( pageCount )  :
    offset = idx * QUERY_FETCH_SIZE
    cursor.execute("SELECT id ,disease, platform_type, center, platform, level, barcode, file_name FROM file_manifest LIMIT %d, %d" %(offset,QUERY_FETCH_SIZE))
    processDBData( cursor, pool )
    
    
mariadb_connection.close()

put rowkey=TCGA-AA-3814-01A-01R-0905-07
put rowkey=TCGA-DM-A0XF-01A-11R-A155-07
put rowkey=TCGA-IC-A6RF-11A-21R-A336-31
put rowkey=TCGA-27-2523-01A-01R-1850-01
put rowkey=TCGA-GR-7351-01A-11R-2213-07
put rowkey=TCGA-L5-A4OJ-01A-11R-A260-31
put rowkey=TCGA-AA-A01V-01A-23R-A083-07
put rowkey=TCGA-JY-A93C-01A-11R-A38D-31
put rowkey=TCGA-L5-A4OG-11A-12R-A260-31
put rowkey=TCGA-LN-A4A9-01A-11R-A28J-31
put rowkey=TCGA-FF-A7CR-01A-11R-A38C-07
put rowkey=TCGA-44-2668-01B-02R-A277-07
put rowkey=TCGA-LN-A9FO-01A-11R-A38D-31
put rowkey=TCGA-28-5209-01A-01R-1850-01
put rowkey=TCGA-FA-A6HN-01A-11R-A31O-07
put rowkey=TCGA-44-2665-01B-06R-A277-07
put rowkey=TCGA-Z6-AAPN-01A-11R-A406-31
put rowkey=TCGA-A7-A26J-01A-11R-A277-07
put rowkey=TCGA-AA-3684-01A-02R-0905-07
put rowkey=TCGA-VR-A8EO-01A-11R-A36D-31
put rowkey=TCGA-AA-A00Z-01A-01R-A002-07
put rowkey=TCGA-06-0680-11A-32R-A36H-07
put rowkey=TCGA-G8-6325-01A-11R-2213-07
put rowkey=TCGA-G4-6303-01A-11R-1774-07
put rowkey=TCGA-06-5410-01A-01R-1849-01
